### Batches, Batch Normalization and Dropout

In this workbook you can experiment what you learnt about how to make batches out of your data, how to perform batch normalization and dropout

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Load the data from data/batches_norm_drop.csv, then take a look at them.

In [2]:
# YOUR CODE HERE
data = pd.read_csv(r'D:\DATASCIENCE\DeepLearning-DS\Chapter 03\04. DataLoader\data\batches_norm_drop.csv', header=None)


In [3]:
data

,0,1,2
0,0.350140,4.248592,0
1,0.950728,3.528855,0
2,1.371517,3.149416,0
3,0.268221,4.337209,0
4,1.881996,1.515387,0
...,...,...,...
745,-3.425455,3.349783,2
746,-1.513002,2.789840,2
747,-1.070356,3.484981,2
748,-2.970848,3.443924,2


In [4]:
x = data.iloc[:,:-1].values
y = data.iloc[:,-1].values

In [5]:
from sklearn.model_selection import train_test_split
import torch

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 0)

x_train = torch.tensor(x_train.astype(np.float32))
x_test = torch.tensor(x_test.astype(np.float32))

y_train = torch.tensor(y_train.astype(np.float32))
y_test = torch.tensor(y_test.astype(np.float32))

Now you are ready to code your own function to create batches. If needed rewatch the video we provided in Eduflow.

**Extra challange:**    Are you able to split between train and test _**without**_ using sklearn?

In [6]:
def batchify(x_train, x_test, y_train, y_test, batch_size):
#YOUR CODE HERE
    n_batches = x_train.shape[0] // batch_size # 11 / 3 = 3.66 -> 3
    n_batches_test = x_test.shape[0] // batch_size

    indexes = np.random.permutation(x_train.shape[0])
    indexes_test = np.random.permutation(x_test.shape[0])


    x_train = x_train[indexes]
    y_train = y_train[indexes]

    x_test = x_test[indexes_test]
    y_test = y_test[indexes_test]

    x_train = x_train[ :batch_size * n_batches ].reshape(n_batches, batch_size, x_train.shape[1])
    y_train = y_train[ :batch_size * n_batches ].reshape(n_batches, batch_size, 1)
    
    x_test = x_test[ :batch_size * n_batches_test ].reshape(n_batches_test, batch_size, x_test.shape[1])
    y_test = y_test[ :batch_size * n_batches_test ].reshape(n_batches_test, batch_size, 1)


    return x_train, x_test, y_train, y_test    


It's time to create your model! Remember to include the new tricks you learnt (batch normalization and dropout)!

In [11]:
# YOUR CODE HERE

import torch.nn.functional as F
from torch import nn
class Classify(nn.Module):
    def __init__(self):
        super(Classify, self).__init__()
        self.input_layer = nn.Linear(2, 56)
        self.batchnorm = nn.BatchNorm1d(56) # applying batch norm 
        self.hidden1 = nn.Linear(56, 49)
        self.drop = nn.Dropout(0.1) 
        self.hidden2 = nn.Linear(49, 8) 
        self.output = nn.Linear(8, 3)
    
    def forward(self, x):
        #first_layer = self.input_layer(x)
        x = self.batchnorm(self.drop(F.relu(self.input_layer(x))))
        #act1 = F.relu(first_layer)
        x = (self.hidden1(x))

        x = self.drop(self.hidden2(x))

        x = self.output(x)
        
        x = F.softmax(x)
        return x


Train your model and evaluate it. **Extra challenge**: try to figure out how you can tell if batch norm and dropout are effective

In [12]:
# YOUR CODE HERE
import torch.optim as optim

model = Classify()

epochs = 100


criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3)

loss_train = []
loss_test = []


for epoch in range(epochs):

    running_train_loss = 0
    running_test_loss = 0

    x_trbh, x_tsbh, y_trbh, y_tsbh = batchify(x_train, x_test, y_train, y_test, 20)
    for i in range(x_trbh.shape[0]):
        

        optimizer.zero_grad()
        pred=model.forward(x_trbh[i])
        train_loss=criterion(pred,y_trbh[i])
        train_loss.backward()
        optimizer.step()


        model.eval()
        with torch.no_grad():

            test_pred = model.forward(x_tsbh)

            test_loss = criterion(test_pred, y_tsbh)


        model.train()

        running_train_loss += train_loss.item()
        running_test_loss += test_loss.item()

        #loss_train.append(train_loss.item())
        #loss_test.append(test_loss.item())
       
        print(f'Epoch: {epoch + 1} | loss: {train_loss.item()} | test loss: {test_loss.item()}' )

    loss_train.append(running_train_loss/len(x_trbh))
    loss_test.append(running_test_loss/len(x_tsbh))

plt.plot(loss_train, label='train Loss')
#plt.plot(loss_test, label='test Loss')

plt.legend()
plt.show()


C:\Anaconda\envs\strive\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Anaconda\envs\strive\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([20, 1])) that is different to the input size (torch.Size([20, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: running_mean should contain 20 elements not 56